In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from preprocess import * 
from predict import *
from utils import *
from torchmetrics import WordErrorRate, CharErrorRate, MatchErrorRate

/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
data_path = "/home/said/courses/Projetinfo/sports_scr/data/champions/natation.txt"
dic_names = get_champions_names(data_path)
scoreboard_path = "../data/natation/ROI/CLIP_visual_prompt/scoreboard.png"
default_visual_prompt = Image.open(scoreboard_path)
default_visual_prompt = np.array(default_visual_prompt)[:,:, 0:3]

In [5]:
def get_ground_truth(path):
    """Returns ground truth data

    Args:
        path (str): path to the ground truth txt file

    Returns:
        list: list of the ground truth data
    """

    with open(path, "r") as f:
        return [line.strip() for line in f.readlines()]
    
def compute_metrics(result, groundtruth):
    wer = WordErrorRate()
    cer = CharErrorRate()

    word_error_rate = 0
    character_error_rate = 0

    for yhat, y in zip(result, groundtruth):
        word_error_rate += wer(yhat, y)
        character_error_rate += cer(yhat, y)

    return word_error_rate/min(len(result), len(groundtruth)), character_error_rate/min(len(result), len(groundtruth))

In [6]:
data_path ="../data/champions/natation.txt"
dic_names = get_champions_names(data_path)

In [32]:
def process_img(img, orientation, clip):
    if clip:
        modified_img = extract_roi_clipseg_visual(img, prompt=default_visual_prompt, thresh=0.5)
        if orientation:
            modified_img = correct_orientation(modified_img)
        return modified_img
    else:
        if orientation:
            modified_img = correct_orientation(modified_img)
            return modified_img
        else:
            return np.array(Image.open(img))

def evaluate(ocr_engine, orientation, clip):
    total_wer, total_cer = 0, 0
    img_paths = create_images_path_list("../data/natation")
    for img in img_paths:
        processed_img = process_img(img, orientation, clip)

        predictions = infer(ocr_engine, processed_img)
        input_string = ' '.join(predictions)
        result = extract_names_scores(input_string, dic_names, min_edit_distance=3)

        groundtruth_path = img.split('.')
        groundtruth_path[-1] = ".txt"
        groundtruth_path = '..' + ''.join(groundtruth_path[2:])
        groundtruth = get_ground_truth(groundtruth_path)

        try:
            wer, cer = compute_metrics(result, groundtruth)
        except:
            wer, cer = 1, 1
        total_wer += wer
        total_cer += cer
    return total_wer/len(img_paths), total_cer/len(img_paths)

# docTR

## Base images

In [33]:
doctr_base_total_wer, doctr_base_total_cer = evaluate(ocr_engine='doctr', orientation=False, clip=False)

## Base + correction d'orientation

In [ ]:
doctr_orient_total_wer, doctr_orient_total_cer = evaluate(ocr_engine='doctr', orientation=True, clip=False)

## CLIP images

## CLIP + correction d'orientation